In [1]:
import pandas as pd
import os
import glob

In [5]:
%pwd

'/content'

In [25]:
path_lst = glob.glob('/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/*.csv')
path_lst

['/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/201112_201112_연령별인구현황_월간.csv',
 '/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/201201_201201_연령별인구현황_월간.csv',
 '/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/201204_201204_연령별인구현황_월간.csv',
 '/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/201205_201205_연령별인구현황_월간.csv',
 '/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/201206_201206_연령별인구현황_월간.csv',
 '/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/201207_201207_연령별인구현황_월간.csv',
 '/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/성별_고령인구_201112_201612/201209_201209_연령별인구현화

1. '전국' 삭제
2. 총인구수, 연령구간인구수 만 사용
3. 행정구역 : 2012 상반기까지 세종 없음 주의
4. 행정구역 : 지역코드 삭제
5. 데이터프레임 형식
- 지역(region) - 연도(yyyy) - 월(mm) - 성별(1,2) - 고령인구수(elderly_pop) - 총 인구수(total_pop) - 고령층비율(elderly_ratio) 

In [44]:
elderly_frm = pd.DataFrame(columns = ['region','yyyy','mm','sex','elderly_pop', 'total_pop'])

mapping_dic = {'강원도':'강원',
           '경기도':'경기',
           '서울특별시':'서울',
           '세종특별자치시':'세종',
           '부산광역시':'부산',
           '대구광역시':'대구',
           '인천광역시':'인천',
           '광주광역시':'광주',
           '대전광역시':'대전',
           '울산광역시':'울산',
           '충청북도':'충북',
           '충청남도':'충남',
           '경상북도':'경북',
           '경상남도':'경남',
           '전라북도':'전북',
           '전라남도':'전남',
           '제주특별자치도':'제주'}

In [45]:
for path in path_lst :
    test = pd.read_csv(path, encoding='cp949', thousands = ',', index_col=0)[1:]
    test.index = test.index.str.split().str[0]
    test01 = test.filter(regex='총인구수|연령구간인구수', axis=1)
    male = test01.filter(like='남', axis=1)
    female = test01.filter(like='여', axis=1)
    for frm in [male, female] :
        yyyy = frm.columns[0].split('_')[0][:4]
        mm   = frm.columns[0].split('_')[0][5:7]
        sex  = frm.columns[0].split('_')[1]
        frm.columns = ['total_pop', 'elderly_pop']
        frm['yyyy'] = yyyy
        frm['mm']   = mm
        frm['sex']  = sex.replace('남','1').replace('여','2')

    concat_frm = pd.concat([male, female])[['yyyy','mm','sex', 'elderly_pop', 'total_pop']].reset_index().rename(columns = {'행정구역':'region'})
    elderly_frm = elderly_frm.append(concat_frm)

# region 두글자 통일
elderly_frm.region = elderly_frm.region.map(mapping_dic)

# region 기준 오름차순 정렬
elderly_frm = elderly_frm.sort_values(by=['region','yyyy','mm']).reset_index(drop=True)

display(elderly_frm.head(10))
print(elderly_frm.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,region,yyyy,mm,sex,elderly_pop,total_pop
0,강원,2011,12,1,95929,773278
1,강원,2011,12,2,137290,763170
2,강원,2012,01,1,96281,772718
3,강원,2012,01,2,137678,762617
4,강원,2012,02,1,96716,772155
5,강원,2012,02,2,138233,762155
6,강원,2012,03,1,97088,773580
7,강원,2012,03,2,138648,762945
8,강원,2012,04,1,97413,773835
9,강원,2012,04,2,139055,763261


(2060, 6)


In [47]:
elderly_frm.to_csv('/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/201112_201612_성별_고령인구.csv', encoding='cp949', index=False)

고령층비율(elderly_ratio) 컬럼 추가 = elderly_pop / total_pop * 100

In [3]:
elderly_frm = pd.read_csv('/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/201112_201612_성별_고령인구.csv', encoding='cp949')
elderly_frm.head()

,region,yyyy,mm,sex,elderly_pop,total_pop
0,강원,2011,12,1,95929,773278
1,강원,2011,12,2,137290,763170
2,강원,2012,1,1,96281,772718
3,강원,2012,1,2,137678,762617
4,강원,2012,2,1,96716,772155


In [7]:
elderly_frm['elderly_ratio'] = (elderly_frm.elderly_pop / elderly_frm.total_pop) * 100
elderly_frm

,region,yyyy,mm,sex,elderly_pop,total_pop,elderly_ratio
0,강원,2011,12,1,95929,773278,12.405500
1,강원,2011,12,2,137290,763170,17.989439
2,강원,2012,1,1,96281,772718,12.460044
3,강원,2012,1,2,137678,762617,18.053361
4,강원,2012,2,1,96716,772155,12.525464
...,...,...,...,...,...,...,...
2055,충북,2016,10,2,139849,787210,17.765145
2056,충북,2016,11,1,99734,802676,12.425188
2057,충북,2016,11,2,140189,787861,17.793621
2058,충북,2016,12,1,100184,803240,12.472486


In [8]:
elderly_frm.to_csv('/content/drive/MyDrive/공모전/날씨 빅데이터 콘테스트/data/201112_201612_성별_고령인구.csv', encoding='cp949', index=False)